<a href="https://colab.research.google.com/github/m-movahedi/sighseeing_tour_planning/blob/master/Sighting_Seeing_Route_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Requirements

In [0]:
import subprocess
import sys

package_list=['ortools','shapely','folium','Fiona','os','zipfile','agent','rlglue','geopandas','openrouteservice']
def install(package):
  for package in package_list:
    try:
      subprocess.check_call([sys.executable, "!pip", "install", package])
    except:
      print(package,' is not installed')
      pass
try:
  !pip install Fiona
  !pip install ortools
  !pip install openrouteservice
  !pip install geopandas
  !pip install folium
except:
  pass
from shapely import wkt, geometry
import json
import folium
import os
from pprint import pprint
import fiona
from google.colab import files
from shapely.geometry import shape, LineString
import zipfile
import geopandas as gpd
import numpy as np
from openrouteservice import geocode
project_name=input("Name of the project?")
os.chdir('/content/')
try:
  os.chdir(project_name)
except:
  try:
    os.mkdir(project_name)
  except:
    pass
  os.chdir (project_name)
cwd = os.getcwd()
try:
  os.mkdir('StudyArea')
except:
  pass
try:
  os.mkdir('Attractions')
except:
  pass
os.chdir(cwd)
try:
  os.mkdir('Outputs')
except:
  pass
os.chdir(cwd)
def upload_file_shapefile_from_zip(cwd,location):
  try:
    os.chdir(location)
    uploaded = files.upload()
    temp_cwd=os.getcwd()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))
    name=[]
    for i in uploaded.keys():
      name.append(i)
    temp=temp_cwd+'/'+name[0]
    #print(temp)
    zip_ref = zipfile.ZipFile(temp, 'r')
    zip_ref.extractall()
    zip_ref.close()
    os.chdir(cwd)
    print(temp[:-4])
    return(temp_cwd,temp_cwd+'/'+name[0][:-4]+'.shp')
  except:
    print("Retry choosing the shapefile")
    os.chdir(cwd)
#Create WTK format coordinations
def wtk_maker(c):
  collection = [ shape(item['geometry']) for item in c ]
  rings = [ LineString(pol.exterior.coords).wkt for pol in collection ]
  coordinations=''
  for coordination in rings[0][14:-1].split(','):
    temp=coordination.split()
    if len(temp)>2:
      coordinations=coordinations+str(temp[0])+' '+str(temp[1])+','
  wkt_str='Polygon (('+coordinations[:-1]+'))'
  return(wkt_str)
  #API key
api_key = 'API KEY'

     |████████████████████████████████| 14.7MB 301kB/s 
     |████████████████████████████████| 27.9MB 32.5MB/s 
     |████████████████████████████████| 1.3MB 42.9MB/s 
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0


     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 10.4MB 40kB/s 
Name of the project?District_11


In [0]:
import folium
from folium.plugins import BeautifyIcon
import shapely
import pandas as pd
import openrouteservice as ors
from IPython.core.interactiveshell import InteractiveShell

#Routes

## Main route
To visit all stations, a route should be planned. As the vehicles meant to carry tourists in this area were antique refurbished buses, the program was set to find a route fit for HGV vehicles.

In [0]:
def Route_finding(mode,path_StudyArea,shp_path_StudyArea,path_Attractions,shp_Attractions):
  mode=mode
  c_Attractions = fiona.open(shp_Attractions)
  data = gpd.read_file(shp_Attractions)
  c_StudyArea = fiona.open(shp_path_StudyArea)
  wkt_str=wtk_maker(c_StudyArea )
  aoi_geom = wkt.loads(wkt_str) 
  aoi_coords = list(aoi_geom.exterior.coords)
  aoi_coords = [(y,x) for x,y in aoi_coords]
  aoi_centroid = aoi_geom.centroid 
  m = folium.Map(tiles='Stamen Toner',location=(aoi_centroid.y, aoi_centroid.x), zoom_start=14)
  folium.vector_layers.Polygon(aoi_coords,color='#ffd699',fill_color='#ffd699',fill_opacity=0.2,weight=3).add_to(m)
  Attractions_addresses = []
  for i in range((data['geometry'].size)-1):
      lon=data['geometry'].iloc[i].x
      lat=data['geometry'].iloc[i].y
      name = data['Name'].iloc[i]
      popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
      icon_url='https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Red_X.svg/100px-Red_X.svg.png'
      icon = folium.features.CustomIcon(icon_url, icon_size=(16,16))
      temp_loc_attractions=[lat, lon]
      folium.map.Marker(temp_loc_attractions, icon=icon, popup=popup).add_to(m)
      Attractions_addresses.append(name)  
  print(len(Attractions_addresses))
  Attraction_coords=[]
  for i in range(data['geometry'].size):
      lon=data['geometry'].iloc[i].x
      lat=data['geometry'].iloc[i].y
      Attraction_coords.append([lon,lat])
  from openrouteservice import client, places
  clnt = client.Client(key=api_key)
  from openrouteservice import distance_matrix
  request = {'locations': Attraction_coords,'profile': mode,'metrics': ['duration']}
  Attraction_matrix = clnt.distance_matrix(**request)
  print("Calculated {}x{} routes.".format(len(Attraction_matrix['durations']),len(Attraction_matrix['durations'][0])))
  def create_data_model():
    """Stores the data for the problem."""
    data_set = {}
    data_set['distance_matrix'] = distance_matrix(Attractions_addresses)
    data_set['Names'] = np.asarray(data['Name'])
    data_set['num_vehicles'] = 1
    data_set['depot'] = 0
    return data_set
  def distance_matrix(Attractions_addresses):
    size=len(Attractions_addresses)
    distance_matrix =[]
    distance_matrix_temp=[]
    for i in range(size):
      for j in range(size):
        if i!=j:
          temp=int(round(Attraction_matrix['durations'][i][j]))
          distance_matrix_temp.append(temp)
        else:
          distance_matrix_temp.append(0)
      distance_matrix.append(distance_matrix_temp)
      distance_matrix_temp=[]
    return(np.asarray(distance_matrix))
  from ortools.constraint_solver import pywrapcp
  from ortools.constraint_solver import routing_enums_pb2
  def getDistance(from_id, to_id):
    return int(Attraction_matrix['durations'][from_id][to_id])
  data_set = create_data_model()
  tsp_size = len(data_set['distance_matrix'])
  num_routes = data_set['num_vehicles']
  start = data_set['depot'] # arbitrary start location
  coords_aoi = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x)

  optimal_coords = []

  if tsp_size > 0:
      manager = pywrapcp.RoutingIndexManager(tsp_size-1,num_routes,start)
      routing = pywrapcp.RoutingModel(manager)
      search_parameters = pywrapcp.DefaultRoutingSearchParameters()
      # Create the distance callback, which takes two arguments (the from and to node indices) and returns the distance between these nodes.
      dist_callback = getDistance
      transit_callback_index = routing.RegisterTransitCallback(dist_callback)
      routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
      # Solve, returns a solution if any.
      assignment = routing.SolveWithParameters(search_parameters)
      if assignment:
          # Total cost of the 'optimal' solution.
          print("Total duration: " + str(round(assignment.ObjectiveValue(), 3) / 60) + " minutes\n")
          index = routing.Start(start) # Index of the variable for the starting node.
          route = ''
         # while not routing.IsEnd(index):
          for node in range(routing.nodes()):
              optimal_coords.append(Attraction_coords[manager.IndexToNode(index)])
              route += str(Attraction_coords[manager.IndexToNode(index)]) + ' -> '
              index = assignment.Value(routing.NextVar(index))
          route += str(Attraction_coords[manager.IndexToNode(index)])
          optimal_coords.append(Attraction_coords[manager.IndexToNode(index)])
          print("Route:\n" + route)
  #optimal_coords.insert(0,Attraction_coords[0])
  import os.path
  def style_function(color):
      return lambda feature: dict(color=color,
                                weight=4,
                                opacity=1)
  # 'random' tour 
  Attraction_coords.append(Attraction_coords[0])
  request = {'coordinates': Attraction_coords[:-2],
            'profile': mode,
            'geometry': 'true',
            'format_out': 'geojson',
  #            'instructions': 'false'          
            }
  random_route = clnt.directions(**request)
  #Changes
  fg_Random_route=folium.FeatureGroup(name='Random Route')
  m.add_child(fg_Random_route)
  #changes:  .add_to(fg_Random_route)
  folium.features.GeoJson(data=random_route,
                          name='Random Crawl',
                          style_function=style_function('#ababab'),
                        overlay=True).add_to(fg_Random_route)
  # optimal route

  #changes:
  fg_optimal_route=folium.FeatureGroup(name='Optimal Route')
  m.add_child(fg_optimal_route)
  #changes: add_to(m) to add_to(fg_optimal_route) 
  request['coordinates'] = optimal_coords
  optimal_route = clnt.directions(**request)
  folium.features.GeoJson(data=optimal_route,
                          name='Optimal Crawl',
                          style_function=style_function('#0090f7'),
                        overlay=True).add_to(fg_optimal_route)

  #m.add_child(folium.map.LayerControl())
  request = {'locations': Attraction_coords,'profile': mode,'metrics': ['distance']}
  Attraction_distance_matrix = clnt.distance_matrix(**request)
  def distance_matrix(Attractions_addresses,Attraction_distance_matrix):
    size=len(Attractions_addresses)
    distance_matrix =[]
    distance_matrix_temp=[]
    for i in range(size):
      for j in range(size):
        if i!=j:
          temp=int(round(Attraction_distance_matrix['distances'][i][j]))
          distance_matrix_temp.append(temp)
        else:
          distance_matrix_temp.append(0)
      distance_matrix.append(distance_matrix_temp)
      distance_matrix_temp=[]
    return(np.asarray(distance_matrix))
  #print("Calculated {}x{} routes.".format(len(Attraction_distance_matrix['distance']),len(Attraction_distance_matrix['distance'][0])))
  return(m,optimal_route,data,data_set['distance_matrix'],distance_matrix(Attractions_addresses,Attraction_distance_matrix))

## Subroute
In some cases, we face a compact, walkable site. For example, a historic street that was turned to a walking street or an old ally. In this scenario, at least one station should be located near this area, and a subroute should be planned to connect these attractions. There might not be a visible sign or painted line that demonstrates this subroute, but this process reveals the interest path to the stakeholders and help them to promote this path. Also, determining the subroutes help us to locate the information signs and stands.

In [0]:
def subroute(start_station_number,final_station_number,data_pedestrian,target_map,route_name):
  start_station_number=start_station_number-1
  final_station_number=final_station_number-1
  start_yx=[stations['geometry'][start_station_number].y,stations['geometry'][start_station_number].x]
  start_xy=[stations['geometry'][start_station_number].x,stations['geometry'][start_station_number].y]
  #try:
  finish_yx=[stations['geometry'][final_station_number].y,stations['geometry'][final_station_number].x]
  finish_xy=[stations['geometry'][final_station_number].x,stations['geometry'][final_station_number].y]
  #except:
  poi_pedestrian=[]
  poi_pedestrian_xy=[]
  for i in data_pedestrian['geometry']:
    poi_pedestrian.append([i.y , i.x])
    poi_pedestrian_xy.append([i.x , i.y])
  all_pedestrian=poi_pedestrian
  all_pedestrian.insert(0,start_yx)
  all_pedestrian.insert(-1,finish_yx)
  poi_pedestrian_xy.insert(0,start_xy)
  poi_pedestrian_xy.insert(-1,finish_xy)
  print(all_pedestrian)    
  #folium.map.Marker(location=[start_yx,finish_yx],icon=folium.Icon(color="green", icon="bus", prefix='fa'),).add_to(target_map)
  for i in range(data_pedestrian['geometry'].size):
      lon=data_pedestrian['geometry'].iloc[i].x
      lat=data_pedestrian['geometry'].iloc[i].y
      name = data_pedestrian['Name'].iloc[i]
      popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
      icon_url='https://cdn2.iconfinder.com/data/icons/geometry-shapes/614/2074_-_Circle-512.png'
      icon = folium.features.CustomIcon(icon_url, icon_size=(20,20))
      folium.map.Marker([lat, lon], icon=icon, popup=popup).add_to(target_map)
      #Attractions_addresses.append(name)  
  import requests
  body1 = {"coordinates":poi_pedestrian_xy,"preference":"shortest"}
  body2=body = {"coordinates":poi_pedestrian_xy,"preference":"fastest"}
  body3 = {"coordinates":poi_pedestrian_xy,"preference":"recommended"}
  headers = {
      'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
      'Authorization': api_key,
      'Content-Type': 'application/json; charset=utf-8'
  }
  call1 = requests.post('https://api.openrouteservice.org/v2/directions/foot-walking/geojson', json=body1, headers=headers)
  call2 = requests.post('https://api.openrouteservice.org/v2/directions/foot-walking/geojson', json=body2, headers=headers)
  call3 = requests.post('https://api.openrouteservice.org/v2/directions/foot-walking/geojson', json=body3, headers=headers)
  calls=[call1,call2,call3]
  colors=['#fa5a95','#a665db','#73cf04']
  names=["shortest","fastest","recommended"]
  def style_function(color):
      return lambda feature: dict(color=color,
                                weight=3,
                                opacity=0.75)
  for idx in range(len(calls)):
    call=calls[idx]
    print(call.status_code, call.reason)
    distance=call.json()['features'][0]['properties']['summary']['distance']
    duration=call.json()['features'][0]['properties']['summary']['duration']
    print('Total length of the',names[idx] ,' route for walking rote ',idx,' is ',distance,' meters which takes',duration/60, 'minutes to visit.')
    layer_name=names[idx]+' route for '+route_name
    fg=folium.FeatureGroup(name=layer_name)
    target_map.add_child(fg)
    color=colors[idx]
    gj = folium.GeoJson(
            data=call.json(),
            name=names[idx],
            style_function=style_function(color)
        )
    gj.add_to(fg)
  #folium.LayerControl().add_to(target_map)
  return(target_map,calls)

## Executing

### Adding The Study Area & Stations
At this step, the study area is being determined. By determined this area, we can search within its borders to find attractions.
A Shapefile containing attractions should be imported. The location of these points is representing the stations that were pinpointed before.

In [0]:
path_StudyArea,shp_path_StudyArea =upload_file_shapefile_from_zip(cwd,'StudyArea')

Saving Study2.zip to Study2.zip
User uploaded file "Study2.zip" with length 1141 bytes
/content/District_11/StudyArea/Study2


Saving Stations_V20.zip to Stations_V20.zip
User uploaded file "Stations_V20.zip" with length 2198 bytes
/content/District_11/Attractions/Stations_V20


In [0]:
path_Attractions,shp_Attractions =upload_file_shapefile_from_zip(cwd,'Attractions')

Saving station_V25.zip to station_V25.zip
User uploaded file "station_V25.zip" with length 2134 bytes
/content/District_11/Attractions/station_V25


### Adding Subroutes
An integrated network of motorized and non-motorized routes can be a more efficient approach to this problem. In this step, the attractions that have such potential is being connected. A shapefile containing the location should be imported. The code of the stations meant for these paths should be determined.
For each set, three different routes will be calculated and will be labeled as the shortest, the fastest, and the recommended.

In [0]:
m, optimal_route,stations,distance_matrix,Attraction_distance_matrix=Route_finding('driving-hgv',path_StudyArea,shp_path_StudyArea,path_Attractions,shp_Attractions)
m,calls3=subroute(9,12,data_pedestrian_1,m,'Pedestrian Route 1')
m,calls1=subroute(11,16,data_pedestrian_2,m,'Pedestrian Route 2')
m,calls2=subroute(7,15,data_pedestrian_3,m,'Pedestrian Route 3')

17
Calculated 18x18 routes.
Total duration: 26.066666666666666 minutes

Route:
[51.39822700000008, 35.65986200000003] -> [51.399365000000046, 35.66682910000003] -> [51.40177380000006, 35.68116730000003] -> [51.402374000000066, 35.68704100000008] -> [51.392851000000064, 35.68853700000005] -> [51.38656300000008, 35.69158900000008] -> [51.40446980000007, 35.69715650000006] -> [51.40555000000006, 35.70115100000004] -> [51.41272200000003, 35.70010800000006] -> [51.412123000000065, 35.697426000000064] -> [51.40999700000003, 35.68641000000008] -> [51.409185000000036, 35.68158100000005] -> [51.406825000000026, 35.66961150000003] -> [51.391703000000064, 35.67512100000005] -> [51.389090800000076, 35.67402430000004] -> [51.38992550000006, 35.66768780000007] -> [51.39822700000008, 35.65986200000003]
[[35.68641000000008, 51.40999700000003], [35.686810500000036, 51.40480160000004], [35.687072700000044, 51.40251930000005], [35.68659870000005, 51.40680020000008], [35.686162800000034, 51.41009930000007

In [0]:
folium.TileLayer(tiles='openstreetmap').add_to(m)
folium.TileLayer(tiles='Stamen Terrain').add_to(m)
folium.TileLayer(tiles='Mapbox Bright').add_to(m)
folium.TileLayer(tiles='stamenwatercolor').add_to(m)
folium.TileLayer(tiles='cartodbpositron').add_to(m)
folium.TileLayer(tiles='cartodbdark_matter').add_to(m)
folium.map.LayerControl().add_to(m)
m

In [0]:
path_pedestrian_1,shp_pedestrian_1 =upload_file_shapefile_from_zip(cwd,'Attractions')
c_pedestrian = fiona.open(shp_pedestrian_1)
data_pedestrian_1 = gpd.read_file(shp_pedestrian_1)

path_pedestrian_2,shp_pedestrian_2 =upload_file_shapefile_from_zip(cwd,'Attractions')
c_pedestrian = fiona.open(shp_pedestrian_2)
data_pedestrian_2 = gpd.read_file(shp_pedestrian_2)

path_pedestrian_3,shp_pedestrian_3 =upload_file_shapefile_from_zip(cwd,'Attractions')
c_pedestrian = fiona.open(shp_pedestrian_3)
data_pedestrian_3 = gpd.read_file(shp_pedestrian_3)

Saving P_1.zip to P_1 (1).zip
User uploaded file "P_1.zip" with length 1561 bytes
/content/District_11/Attractions/P_1


Saving P_2.zip to P_2 (1).zip
User uploaded file "P_2.zip" with length 1381 bytes
/content/District_11/Attractions/P_2


Saving P_3.zip to P_3 (1).zip
User uploaded file "P_3.zip" with length 1524 bytes
/content/District_11/Attractions/P_3


# Export

## HTML file

In [0]:
#os.chdir('Outputs')
#export interactive HTML file
Temp_export=project_name+'.html'
m.save(Temp_export)

## The Code itself

In [0]:
!pip install nbconvert

In [0]:
nbconvertp HTML Sighting_Seeing_Route_Planning.ipynb

## The sistance matrix

In [0]:
Attraction_distance_matrix

array([[   0, 2397, 2660, 3577, 4476, 4677, 5467, 5549, 5670, 3767, 2796,
        3091, 2396, 3414, 4196,  781, 3416],
       [3073,    0,  263, 2240, 3140, 4653, 5443, 5525, 5646, 3743, 4115,
        3067, 2372, 1017, 4172, 2423, 3392],
       [2810, 2301,    0, 1977, 2876, 4390, 5180, 5262, 5383, 3480, 3852,
        2804, 2109,  754, 3909, 2160, 3129],
       [5825, 2393, 2656,    0,  899, 2659, 3449, 3531, 3652, 4194, 3943,
        2896, 2200, 3410, 2178, 5381, 3220],
       [5945, 2513, 2776,  960,    0, 2879, 3669, 3750, 3871, 3692, 4063,
        3016, 2321, 3530, 2397, 5501, 3341],
       [6108, 5762, 6025, 4020, 4145,    0,  790, 1094, 2337, 2879, 4227,
        3132, 3749, 6779, 1830, 5665, 3504],
       [5318, 4971, 5234, 3230, 3355, 1521,    0,  304, 1547, 2088, 3436,
        2342, 2959, 5989, 1040, 4875, 2714],
       [5014, 4668, 4931, 2926, 3051, 1218, 1547,    0, 1243, 1785, 3133,
        2038, 2655, 5685,  736, 4571, 2410],
       [3772, 3240, 3503, 1937, 2836, 2635, 3425